In [4]:
import albumentations as A 
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
annotations=[os.path.join("data/obj",i) for i in os.listdir("data/obj") if i.endswith(".txt")]

In [6]:
annotations

['data/obj/IMG_3092.txt',
 'data/obj/IMG_3132.txt',
 'data/obj/msg465090405-8671.txt',
 'data/obj/6Z2_5723.txt',
 'data/obj/6Z2_5508.txt',
 'data/obj/IMG_3031.txt',
 'data/obj/IMG_3133.txt',
 'data/obj/6Z2_4622.txt',
 'data/obj/6Z2_5672.txt',
 'data/obj/6Z2_4803.txt',
 'data/obj/msg465090405-8679.txt',
 'data/obj/6Z2_5528.txt',
 'data/obj/6Z2_4853.txt',
 'data/obj/6Z2_4834.txt',
 'data/obj/IMG_3062.txt',
 'data/obj/6Z2_4811.txt',
 'data/obj/6Z2_5716.txt',
 'data/obj/6Z2_5472.txt',
 'data/obj/6Z2_5000.txt',
 'data/obj/IMG_3131.txt',
 'data/obj/6Z2_4893.txt',
 'data/obj/6Z2_4817.txt',
 'data/obj/msg465090405-8680.txt',
 'data/obj/6Z2_4946.txt',
 'data/obj/6Z2_5109.txt',
 'data/obj/6Z2_5480.txt',
 'data/obj/IMG_3032.txt',
 'data/obj/6Z2_5709.txt',
 'data/obj/6Z2_4764.txt',
 'data/obj/6Z2_5275.txt',
 'data/obj/6Z2_5650.txt',
 'data/obj/IMG_3044.txt',
 'data/obj/IMG_3113.txt',
 'data/obj/IMG_3028.txt',
 'data/obj/IMG_3064.txt',
 'data/obj/6Z2_5739.txt',
 'data/obj/IMG_3084.txt',
 'data/obj/

In [7]:
transform = A.Compose([
            A.Affine(scale=(0.5,1.5),
                    translate_percent={'x': (0, 0.25), 'y': (0, 0.25)},
                    rotate=(-180, 180),
                    shear={'x': (-10,10), 'y': (-10,10)},
                    p=0.3),
            A.GaussianBlur(blur_limit=(3,9), sigma_limit=(0.1, 5),p=0.4),
            A.RandomBrightnessContrast(p=0.3),
            A.RandomSizedBBoxSafeCrop(300,300,p=0.3),
            A.GaussNoise(p=0.3),
            A.HorizontalFlip(p=0.3)],
    
            bbox_params=A.BboxParams(format='yolo'))

In [8]:
class_label=["badge"]

In [ ]:
for i in tqdm(range(7)):
        
    for gt in annotations:
        with open(gt,"r") as f:
            coords=[i.strip().split(" ") for i in f.readlines()]
            # coords=np.array(coords).astype("float")
            # bboxes=[]
            # class_labels=[i[0] for i in f.readlines()]
            coordinates=[list(map(float,i[1:]))+[int(i[0])] for i in coords]
            img_name=gt.replace(".txt",".jpg")
            img=cv2.imread(img_name)
            transformed = transform(image=img,bboxes=coordinates)
            transformed_image = transformed['image']
            transformed_bboxes = transformed['bboxes']
            name=os.path.join("augmented_stacked_images",f"{i}_{os.path.basename(gt)}")
            img_name=name.replace(".txt",".jpg")

            with open(name,"w") as f:
                for cords in transformed_bboxes:
                    f.write(f"{cords[-1]} {cords[0]} {cords[1]} {cords[2]} {cords[3]}\n")
                    #  transformed_labels=transformed["class_labels"]
                    # H,W,C=transformed_image.shape
                    # x,y,w,h=cords[:4]*np.array([W,H,W,H])
                    # x_min,y_min,x_max,y_max=int(x-w/2),int(y-h/2),int(x+w/2),int(y+h/2)
                    # label=class_label[int(cords[-1])]

            cv2.imwrite(img_name,transformed_image)

    #     cv2.rectangle(transformed_image,(x_min,y_min),(x_max,y_max),(255,0,0),2)
    # plt.imshow(transformed_image)
    # plt.show()